In [0]:
!wget https://github.com/endgameinc/ember/archive/master.zip
!unzip master.zip
!rm master.zip
!cp -r ember-master/* .
!rm -r ember-master
!pip install -r requirements.txt
!python setup.py install

--2020-04-05 03:52:32--  https://github.com/endgameinc/ember/archive/master.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/endgameinc/ember/zip/master [following]
--2020-04-05 03:52:32--  https://codeload.github.com/endgameinc/ember/zip/master
Resolving codeload.github.com (codeload.github.com)... 140.82.112.10
Connecting to codeload.github.com (codeload.github.com)|140.82.112.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip’

master.zip              [   <=>              ]  11.22M  12.9MB/s    in 0.9s    

2020-04-05 03:52:33 (12.9 MB/s) - ‘master.zip’ saved [11769023]

Archive:  master.zip
04c37efb20ebeb7b99d8ec1d5cdd20ab0328bb36
   creating: ember-master/
  inflating: ember-master/LICENSE.txt  
  inflating: ember-master/README.md  
   creatin

In [0]:
def make_model():
  import tensorflow as tf
  from tensorflow import keras
  from tensorflow.keras import layers
  feature_size=2381
  tf.compat.v1.disable_eager_execution()

  keras.backend.clear_session()
  
  #Model architecture
  from tensorflow.keras import layers
  
  model = tf.keras.Sequential()
  model.add(layers.InputLayer(input_shape=(1,feature_size)))
  model.add(layers.Dropout(0.2))
  model.add(layers.Dense(1500, activation='relu',activity_regularizer=tf.keras.regularizers.l1(l=0.01)))
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(1, activation='sigmoid'))

  model.compile(tf.keras.optimizers.Adam(learning_rate=0.001),
            loss='binary_crossentropy',
            metrics=['accuracy',tf.keras.metrics.AUC(),tf.keras.metrics.Precision()])
  print(model.summary())
  return model

In [0]:
model = make_model()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout (Dropout)            (None, 1, 2381)           0         
_________________________________________________________________
dense (Dense)                (None, 1, 1500)           3573000   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 1500)           0         
_________________________________________________________________
dense_1 (Dense)              (None, 1, 1)              1501      
Total params: 3,574,501
Trainable params: 3,574,501
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
!mkdir ember_2017_2
!cp -r drive/My\ Drive/data/ember_2017_2/* ember_2017_2/

In [0]:
import ember
X_train, y_train, X_test, y_test = ember.read_vectorized_features("ember_2017_2/")
metadata_dataframe = ember.read_metadata("ember_2017_2/")

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [0]:
labelrows = (y_train != -1)

In [0]:
X_train = X_train[labelrows]

In [0]:
y_train = y_train[labelrows]

In [0]:
len(y_train)

600000

In [0]:
import h5py
h5f = h5py.File('X_train.h5', 'w')
h5f.create_dataset('X_train', data=X_train)
h5f.close()

In [0]:
import h5py
h5f = h5py.File('y_train.h5', 'w')
h5f.create_dataset('y_train', data=y_train)
h5f.close()

In [0]:
!rm drive/My\ Drive/data/ember_2017_2/X_train.dat
!rm drive/My\ Drive/data/ember_2017_2/y_train.dat
!cp X_train.h5 drive/My\ Drive/data/ember_2017_2/
!cp y_train.h5 drive/My\ Drive/data/ember_2017_2/

In [0]:
from sklearn.preprocessing import StandardScaler
mms = StandardScaler()

In [0]:
for x in range(0,600000,100000):
  mms.partial_fit(X_train[x:x+100000])

In [0]:
with open("drive/My Drive/data/pickles/mms_scaler","wb") as f:
  import pickle
  pickle.dump(mms,f)
  f.close()

In [0]:
import h5py
h5f = h5py.File('X_train.h5','r')
X_train = h5f['X_train'][:]
h5f.close()

In [0]:
X_train = mms.transform(X_train)

In [0]:
import numpy as np
X_train = np.reshape(X_train,(-1,1,2381))
y_train = np.reshape(y_train,(-1,1,1))

In [0]:
%tensorflow_version 2.x
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

pkl_base="drive/My Drive/data/pickles/"

model.compile(tf.keras.optimizers.Adam(learning_rate=0.01),
          loss='binary_crossentropy',
          metrics=['accuracy',tf.keras.metrics.AUC(),tf.keras.metrics.Precision()])

history = model.fit(X_train, y_train,
                batch_size=256,
                epochs=1,
                  validation_data =((X_train[600000-120000:600000],y_train[600000-120000:600000]))
                  )
model_name="my_model_test-tpu3.h5"
model_weights="weights_test-tpu3.h5"
model.save(pkl_base+model_name)
model.save_weights(pkl_base+model_weights)
print(model_name,model_weights," are saved.")

Train on 600000 samples, validate on 120000 samples
600000/600000 [==============================] - 232s 387us/sample - loss: 7.2819 - accuracy: 0.9602 - auc_2: 0.9656 - precision_2: 0.9604 - val_loss: 2.2820 - val_accuracy: 0.9869 - val_auc_2: 0.9901 - val_precision_2: 0.9884
my_model_test-tpu3.h5 weights_test-tpu3.h5  are saved.


In [0]:
%tensorflow_version 2.x
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

pkl_base="drive/My Drive/data/pickles/"

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

model.compile(tf.keras.optimizers.Adam(learning_rate=0.01),
          loss='binary_crossentropy',
          metrics=['accuracy',tf.keras.metrics.AUC(),tf.keras.metrics.Precision()])

history = model.fit(X_train, y_train,
                batch_size=128,
                epochs=30,
                  validation_split=.2,
                  callbacks=[callback]
                  )
model_name="my_model_test-tpu6.h5"
model_weights="weights_test-tpu6.h5"
model.save(pkl_base+model_name)
model.save_weights(pkl_base+model_weights)
print(model_name,model_weights," are saved.")

Train on 480000 samples, validate on 120000 samples
Epoch 1/30
480000/480000 [==============================] - 240s 499us/sample - loss: 7.1064 - accuracy: 0.6970 - auc_1: 0.8024 - precision_1: 0.8299 - val_loss: 8.1311 - val_accuracy: 0.7718 - val_auc_1: 0.8786 - val_precision_1: 0.9880
Epoch 2/30
480000/480000 [==============================] - 237s 493us/sample - loss: 5.2074 - accuracy: 0.6999 - auc_1: 0.8032 - precision_1: 0.8625 - val_loss: 6.2345 - val_accuracy: 0.7967 - val_auc_1: 0.8845 - val_precision_1: 0.9917
Epoch 3/30
480000/480000 [==============================] - 237s 494us/sample - loss: 4.5177 - accuracy: 0.7078 - auc_1: 0.8124 - precision_1: 0.8805 - val_loss: 6.6920 - val_accuracy: 0.7919 - val_auc_1: 0.9196 - val_precision_1: 0.9927
Epoch 4/30
480000/480000 [==============================] - 240s 500us/sample - loss: 3.2733 - accuracy: 0.7008 - auc_1: 0.8018 - precision_1: 0.9309 - val_loss: 16.8144 - val_accuracy: 0.8343 - val_auc_1: 0.9236 - val_precision_1: 0.